In [1]:
import scipy.ndimage
import numpy as np
import os 
import time
import nibabel
from scipy.linalg import norm 
import torch
import matplotlib.pylab as plt

In [2]:
!nvidia-smi

Tue Mar  7 14:10:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.39.01    Driver Version: 510.39.01    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P4000        Off  | 00000000:3B:00.0  On |                  N/A |
| 46%   38C    P8     6W / 105W |    524MiB /  8192MiB |      4%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:AF:00.0 Off |                  N/A |
| 25%   

In [4]:
import torch.nn as nn
import torch.nn.functional as F
print(torch.cuda.is_available())

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
torch.backends.cudnn.benchmark=True

True
cuda:0


In [5]:
class JawsModel2(nn.Module):
    def __init__(self):
        super(JawsModel2, self).__init__()
        self.conv0a = nn.Conv3d(1, 12, 3, padding=1)
        self.conv0b = nn.Conv3d(12, 12, 3, padding=1)
        self.bn0a = nn.BatchNorm3d(12)

        self.ma1 = nn.MaxPool3d(2)
        self.conv1a = nn.Conv3d(12, 12, 3, padding=1)
        self.conv1b = nn.Conv3d(12, 12, 3, padding=1)
        self.bn1a = nn.BatchNorm3d(12)

        self.ma2 = nn.MaxPool3d(2)
        self.conv2a = nn.Conv3d(12, 16, 3, padding=1)
        self.conv2b = nn.Conv3d(16, 16, 3, padding=1)
        self.bn2a = nn.BatchNorm3d(16)

        self.ma3 = nn.MaxPool3d(2)
        self.conv3a = nn.Conv3d(16, 32, 3, padding=1)
        self.conv3b = nn.Conv3d(32, 24, 3, padding=1)
        self.bn3a = nn.BatchNorm3d(24)

        # up

        self.conv2u = nn.Conv3d(24, 16, 3, padding=1)
        self.bn2u = nn.BatchNorm3d(16)
        self.conv2v = nn.Conv3d(16+0*16, 16, 3, padding=1)

        # up

        self.conv1u = nn.Conv3d(16, 12, 3, padding=1)
        self.bn1u = nn.BatchNorm3d(12)
        self.conv1v = nn.Conv3d(12+0*12, 12, 3, padding=1)

        # up

        self.conv0u = nn.Conv3d(12, 12, 3, padding=1)
        self.bn0u = nn.BatchNorm3d(12)
        self.conv0v = nn.Conv3d(12+0*12, 12, 3, padding=1)

        self.conv1x = nn.Conv3d(12, 3, 1, padding=0)

    def forward(self, x):
        x = torch.relu(self.conv0a(x))
        self.li0 = x = torch.relu(self.bn0a(self.conv0b(x)))

        x = self.ma1(x)
        x = torch.relu(self.conv1a(x))
        self.li1 = x = torch.relu(self.bn1a(self.conv1b(x)))

        x = self.ma2(x)
        x = torch.relu(self.conv2a(x))
        self.li2 = x = torch.relu(self.bn2a(self.conv2b(x)))

        x = self.ma3(x)
        x = torch.relu(self.conv3a(x))
        self.li3 = x = torch.relu(self.bn3a(self.conv3b(x)))

        x = F.interpolate(x, scale_factor=2, mode="nearest")

        x = torch.relu(self.bn2u(self.conv2u(x)))

        x = x + self.li2
        x = torch.relu(self.conv2v(x))

        self.lo1 = x
        x = F.interpolate(x, scale_factor=2, mode="nearest")

        x = torch.relu(self.bn1u(self.conv1u(x)))

        x = x + self.li1
        #with concatenation istead of add, it would be "x = torch.cat([x, self.li1], axis=1)", but the number of kernels must be changed in the definition
        x = torch.relu(self.conv1v(x))

        x = F.interpolate(x, scale_factor=2, mode="nearest")
        self.la1 = x

        x = torch.relu(self.bn0u(self.conv0u(x)))

        x = x + self.li0
        x = torch.relu(self.conv0v(x))

        self.out = x = self.conv1x(x)
        x = torch.sigmoid(x)
        return x

In [6]:
net4 = JawsModel2().to(device)
net4_path = "myworking_model4_ver2.pt"
net4.load_state_dict(torch.load(net4_path))
#net2.eval


<All keys matched successfully>

In [10]:
#mesuring the maximum jaws muscle cross-sectional area on both sides 
slice_vol = np.zeros((len(ids),2,2),)
roop_time = time.time()
start_time = time.time()
mm = 1
print(f"slice wide of one side  = {mm}mm")


for n ,subjname in enumerate(ids[:]):
    #print(subjname)
    max2 = np.zeros((2),)
    max4 = np.zeros((2),)
    filename = "../mri/applyjaws_oldmodel/%s_seg.nii.gz" %subjname
    
    data = np.asarray(nibabel.load(filename).dataobj)
    img = nibabel.load(filename)
    vol = np.abs(np.linalg.det(img.affine))
    #print(data.shape)
    #print(img.shape)
    #print(vol)
    c = mmcentpoints2[n,:,:3]
    v = np.cross(c[1] - c[0], c[2] - c[0])
    v /= np.linalg.norm(v)

    pts = np.indices(data.shape).reshape(3, -1).T
    pts = np.hstack((pts,np.ones((pts.shape[0],1))))
    pts = pts @ img.affine.T
    #print(pts)
    #print(pts.shape)
    cpts = ( pts[:,:3]- c[0] )
    
    output = np.dot(cpts, v)
    #output = np.abs(output)
    output = output.reshape(data.shape)
    #print(output.shape)

    
    for i in range(100):
        
        #print(data[(output > i-51) & (output < i-49)].shape)       
        #print(np.hstack((np.bincount(data[(output > i-51) & (output < i-49)].ravel()),np.zeros((5)))))
        label_count = np.hstack((np.bincount(data[(output > i-51) & (output < i-49)].ravel()),np.zeros((5))))
        
        #print(f"now is {i}")
        #print(label_count)
        #print(max2[0]*vol)
        #print(label_count[2]*vol)
        #print(max2[0] < label_count[2]*vol)
        #plane_sum[n,] = label_count[:5]*vol
    
        if max2[0] < label_count[2]*vol:
            max2[0] = label_count[2]*vol
            max2[1] = i
    
        if max4[0] < label_count[4]*vol:
            max4[0] = label_count[4]*vol
            max4[1] = i
            
    slice_vol[n,0,:] = np.array(max2)
    slice_vol[n,1,:] = np.array(max4)        
        
    #print(np.bincount(data.ravel()))
    
    if n%20 == 0:
        print(f"{n} {subjname} is end ; {time.ctime()}, ({time.time() - roop_time :.3f}s) /TOTAL TIME IS {time.time() - start_time :.3f}s")
        roop_time = time.time()
    


slice wide of one side  = 1mm
0 ehime_ehi0010 is end ; Tue Feb 14 18:23:26 2023, (3.625s) /TOTAL TIME IS 3.625s
20 ehime_ehi0105 is end ; Tue Feb 14 18:24:27 2023, (61.006s) /TOTAL TIME IS 64.631s
40 ehime_ehi0236 is end ; Tue Feb 14 18:25:27 2023, (60.384s) /TOTAL TIME IS 125.015s
60 ehime_ehi0415 is end ; Tue Feb 14 18:26:28 2023, (60.397s) /TOTAL TIME IS 185.413s
80 ehime_ehi0594 is end ; Tue Feb 14 18:27:29 2023, (60.807s) /TOTAL TIME IS 246.220s
100 ehime_ehi0726 is end ; Tue Feb 14 18:28:30 2023, (61.813s) /TOTAL TIME IS 308.033s
120 ehime_ehi0863 is end ; Tue Feb 14 18:29:35 2023, (64.387s) /TOTAL TIME IS 372.421s
140 ehime_ehi0981 is end ; Tue Feb 14 18:30:37 2023, (62.063s) /TOTAL TIME IS 434.484s
160 ehime_ehi1155 is end ; Tue Feb 14 18:31:37 2023, (60.178s) /TOTAL TIME IS 494.662s


/tmp/ipykernel_15119/3596930312.py:42: RuntimeWarning: invalid value encountered in greater
  label_count = np.hstack((np.bincount(data[(output > i-51) & (output < i-49)].ravel()),np.zeros((5))))
/tmp/ipykernel_15119/3596930312.py:42: RuntimeWarning: invalid value encountered in less
  label_count = np.hstack((np.bincount(data[(output > i-51) & (output < i-49)].ravel()),np.zeros((5))))


180 ehime_ehi1328 is end ; Tue Feb 14 18:32:37 2023, (60.376s) /TOTAL TIME IS 555.039s
200 hirosaki_H40010 is end ; Tue Feb 14 18:33:38 2023, (60.926s) /TOTAL TIME IS 615.965s
220 hirosaki_H40083 is end ; Tue Feb 14 18:34:41 2023, (62.561s) /TOTAL TIME IS 678.527s
240 hirosaki_H40162 is end ; Tue Feb 14 18:35:44 2023, (62.859s) /TOTAL TIME IS 741.386s
260 hirosaki_H40221 is end ; Tue Feb 14 18:36:47 2023, (63.497s) /TOTAL TIME IS 804.885s
280 hirosaki_H40283 is end ; Tue Feb 14 18:37:48 2023, (60.794s) /TOTAL TIME IS 865.679s
300 hirosaki_H40346 is end ; Tue Feb 14 18:38:51 2023, (62.779s) /TOTAL TIME IS 928.458s
320 hirosaki_H40388 is end ; Tue Feb 14 18:39:55 2023, (63.666s) /TOTAL TIME IS 992.125s
340 hirosaki_H40432 is end ; Tue Feb 14 18:40:57 2023, (62.962s) /TOTAL TIME IS 1055.087s
360 hirosaki_H40474 is end ; Tue Feb 14 18:41:59 2023, (61.648s) /TOTAL TIME IS 1116.735s
380 hirosaki_H40539 is end ; Tue Feb 14 18:43:00 2023, (61.019s) /TOTAL TIME IS 1177.754s
400 hirosaki_H40598 

2000 kumamoto_KU121047 is end ; Tue Feb 14 20:00:31 2023, (53.327s) /TOTAL TIME IS 5828.183s
2020 kumamoto_KU122037 is end ; Tue Feb 14 20:01:21 2023, (49.959s) /TOTAL TIME IS 5878.142s
2040 kumamoto_KU124017 is end ; Tue Feb 14 20:02:10 2023, (49.718s) /TOTAL TIME IS 5927.860s
2060 kyushu_10589 is end ; Tue Feb 14 20:03:04 2023, (53.899s) /TOTAL TIME IS 5981.759s
2080 kyushu_11219 is end ; Tue Feb 14 20:04:03 2023, (59.318s) /TOTAL TIME IS 6041.077s
2100 kyushu_2500 is end ; Tue Feb 14 20:05:06 2023, (62.202s) /TOTAL TIME IS 6103.280s
2120 kyushu_3091 is end ; Tue Feb 14 20:06:07 2023, (61.124s) /TOTAL TIME IS 6164.404s
2140 kyushu_3533 is end ; Tue Feb 14 20:07:06 2023, (59.564s) /TOTAL TIME IS 6223.969s
2160 kyushu_3783 is end ; Tue Feb 14 20:08:06 2023, (60.130s) /TOTAL TIME IS 6284.100s
2180 kyushu_3967 is end ; Tue Feb 14 20:09:06 2023, (59.251s) /TOTAL TIME IS 6343.352s
2200 kyushu_4239 is end ; Tue Feb 14 20:10:06 2023, (60.327s) /TOTAL TIME IS 6403.679s
2220 kyushu_4423 is end

In [13]:
n = 2540
subjname = ids[n]
print(subjname)
i2 = slice_vol[n,0,1]
i4 = slice_vol[n,1,1]
print(i2)
print(i4)
    
filename = "../mri/applyjaws_oldmodel/%s_seg.nii.gz" %subjname
    
data = np.asarray(nibabel.load(filename).dataobj)
data2 = data.copy()
data4 = data.copy()
img = nibabel.load(filename)

#print(data.shape)
#print(img.shape)
c = mmcentpoints2[n,:,:3]
v = np.cross(c[1] - c[0], c[2] - c[0])
v /= np.linalg.norm(v)

pts = np.indices(data.shape).reshape(3, -1).T
pts = np.hstack((pts,np.ones((pts.shape[0],1))))
pts = pts @ img.affine.T
#print(pts)
#print(pts.shape)
cpts = ( pts[:,:3]- c[0] )

output = np.dot(cpts, v)
#print(output.shape)
output = output.reshape(data.shape)
#print(output.shape)
#data[output > mm] = 0
data2[(output < i2-51) | (output > i2-49)] = 0
data4[(output < i4-51) | (output > i4-49)] = 0

nibabel.Nifti1Image(data2, img.affine).to_filename("../mri/applyjaws_oldmodel/%s_max2slice.nii.gz" %subjname)
nibabel.Nifti1Image(data4, img.affine).to_filename("../mri/applyjaws_oldmodel/%s_max4slice.nii.gz" %subjname)

kyushu_9801
36.0
43.0
